### 41. Calcular el sentimiento promedio de cada categoría en base al promedio ponderado del sentimiento de sus géneros. Si por ej una categoría tiene género A y B, A tiene asociado 1 y B asociado -1, si A tiene el 75% y B el 25%, la categoría tiene un sentimiento promedio de 0.5 ⭐⭐

In [18]:
import pandas as pd

In [19]:
apps = pd.read_csv('googleplaystore.csv')
apps.drop(10472, inplace=True)
reviews = pd.read_csv('googleplaystore_user_reviews.csv')

In [20]:
df = apps[['App', 'Genres']]
df.Genres = df.Genres.map(lambda x: list(x.split(';')))
df = df.explode('Genres')
df.head()

/var/folders/k8/xbcdph9j4bdg149mdybx9_x00000gn/T/ipykernel_28794/605421664.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Genres = df.Genres.map(lambda x: list(x.split(';')))


,App,Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,Art & Design
1,Coloring book moana,Art & Design
1,Coloring book moana,Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",Art & Design
3,Sketch - Draw & Paint,Art & Design


In [21]:
sentiment_generos = pd.merge(reviews, df, how='inner', on='App')[['Sentiment_Polarity', 'Genres']].groupby('Genres').mean().reset_index()
sentiment_generos.head()

,Genres,Sentiment_Polarity
0,Action,0.088536
1,Action & Adventure,0.156458
2,Adventure,0.055656
3,Arcade,0.024998
4,Art & Design,0.226257


In [22]:
apps = apps[['App', 'Category', 'Genres']]
apps['Genres'] = apps.Genres.map(lambda x: list(x.split(';')))
apps = apps.explode('Genres')
apps.head()

,App,Category,Genres
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,Art & Design
1,Coloring book moana,ART_AND_DESIGN,Art & Design
1,Coloring book moana,ART_AND_DESIGN,Pretend Play
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,Art & Design
3,Sketch - Draw & Paint,ART_AND_DESIGN,Art & Design


In [23]:
df = apps.groupby('Category')['Genres'].value_counts().reset_index()
df['peso'] = df['count'] / df.groupby('Category')['count'].transform('sum')
df.head()

,Category,Genres,count,peso
0,ART_AND_DESIGN,Art & Design,65,0.902778
1,ART_AND_DESIGN,Creativity,5,0.069444
2,ART_AND_DESIGN,Action & Adventure,1,0.013889
3,ART_AND_DESIGN,Pretend Play,1,0.013889
4,AUTO_AND_VEHICLES,Auto & Vehicles,85,1.000000


In [24]:
df2 = pd.merge(df, sentiment_generos)
df2['polaridad_ponderada'] = df2.peso * df2.Sentiment_Polarity
df2.head()

,Category,Genres,count,peso,Sentiment_Polarity,polaridad_ponderada
0,ART_AND_DESIGN,Art & Design,65,0.902778,0.226257,0.204259
1,FAMILY,Art & Design,4,0.001674,0.226257,0.000379
2,ART_AND_DESIGN,Creativity,5,0.069444,0.103640,0.007197
3,COMICS,Creativity,1,0.016393,0.103640,0.001699
4,EDUCATION,Creativity,3,0.016393,0.103640,0.001699


In [25]:
df2.groupby('Category')['polaridad_ponderada'].sum().reset_index()

,Category,polaridad_ponderada
0,ART_AND_DESIGN,0.215249
1,AUTO_AND_VEHICLES,0.348683
2,BEAUTY,0.193360
3,BOOKS_AND_REFERENCE,0.226526
4,BUSINESS,0.247749
5,COMICS,0.451262
6,COMMUNICATION,0.182965
7,DATING,0.164817
8,EDUCATION,0.219799
9,ENTERTAINMENT,0.140297
